In [1]:
import syft as sy
from syft import ActionObject
from syft import syft_function, syft_function_single_use
from time import sleep
import os
import psutil
import inspect

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/pysyft


with server

In [2]:
node = sy.orchestra.launch(name="test-domain-helm2", dev_mode=True,
                           reset=True, 
                           n_consumers=3,
                           create_producer=True,
                           queue_port=3322)
                           
client = node.login(email="info@openmined.org", password="changethis")

Staging Protocol Changes...
Logged into <test-domain-helm2: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [3]:
res = client.register(name="a", email="aa@b.org", password="c", password_verify="c")

In [4]:
ds_client = node.login(email="aa@b.org", password="c")

Logged into <test-domain-helm2: High side Domain> as <aa@b.org>


setup: compute train-test overlap between a very large train set and a smaller test set. Small test is still to big for memory, so we split it into 54 parts. We keep 1 of those parts in memory. We dont keep the train set in memory, but read and compare with 1/54 parts line by line. Each part takes ~30 hours, but we can run 54 processes in parallel.

In [5]:
# client.rrr()

# Setup syft functions

## Dataset

In [6]:
x = ActionObject.from_obj([1, 2])
x_ptr = x.send(ds_client)

## Batch function

In [7]:
@syft_function()
def process_batch(batch):
    # takes 30 hours normally
    print(f"starting batch {batch}")
    from time import sleep
    [1,2,3,4][5]
    sleep(1)
    print("done")
    return batch+1

SyftSuccess: Syft function 'process_batch' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [8]:
client.code.submit(process_batch)

SyftSuccess: User Code Submitted

## Main function

In [9]:
@syft_function_single_use(x=x_ptr)
def process_all(domain, x):
    
    jobs = []
    print("Launching jobs")
    for elem in x:
        # We inject a domain object in the scope
        batch_job = domain.launch_job(process_batch, batch=elem)
        jobs += [batch_job]
    print("starting aggregation")
    print("Done")
#     results = [x.wait().get() for x in jobs]
    return 3
#     return sum(results)

SyftSuccess: Syft function 'process_all' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

# Approve & run

In [10]:
ds_client.code.request_code_execution(process_all)
client.requests[-1].approve()

Request approved for domain test-domain-helm2


SyftSuccess: Request f994ae2d641a458a8e92ab92bc512b4d changes applied

In [11]:
job = ds_client.code.process_all(x=x_ptr, blocking=False)
sleep(5)

16/11/23 17:40:41 FUNCTION LOG (7406fef2f273425fbc7de2d90739c656): Launching jobs
16/11/23 17:40:41 EXCEPTION LOG (7406fef2f273425fbc7de2d90739c656):

Encountered while executing process_batch:
Traceback (most recent call last):
  File "/Users/koen/workspace/PySyft/packages/syft/src/syft/service/code/user_code.py", line 1072, in execute_byte_code
    result = eval(evil_string, _globals, _locals)  # nosec
  File "<string>", line 1, in <module>
  File "<string>", line 12, in user_func_process_all_0db921031bdea2fe1f9cec8aaa2d399700ebfc3783f5014f83420636e95512dd_cc13b7dc96d28ffe8b072ba1412553fa2db9d76d916d7384b78705c8a61465bb
  File "<string>", line 7, in process_all
NameError: name 'process_batch' is not defined

    4      print('Launching jobs')
    5      for elem in x:
--> 6          batch_job = domain.launch_job(process_batch, batch=elem)
    7          jobs += [batch_job]


In [12]:
job.subjobs

[]

In [12]:
job.subjobs[0].logs(stderr=True)

IndexError: list index out of range

starting batch 2


Encountered while executing process_batch:
Traceback (most recent call last):
  File "/Users/koen/workspace/PySyft/packages/syft/src/syft/service/code/user_code.py", line 1072, in execute_byte_code
    result = eval(evil_string, _globals, _locals)  # nosec
  File "<string>", line 1, in <module>
  File "<string>", line 10, in user_func_process_batch_47f8c8f3db3a30695a28e4a51e44916669ac3d111924cb614181c64b2c3b8323_3fa7f836b5a65455ef117b87f10f6a3f711d956132679e32522b6ae4c1440ec5
  File "<string>", line 6, in process_batch
IndexError: list index out of range

    3      print(f'starting batch {batch}')
    4      from time import sleep
--> 5      [1, 2, 3, 4][5]
    6      sleep(1)


In [10]:
job.wait()

Pointer:
None

In [11]:
sum([j.wait().get() for j in job.subjobs])

TypeError: unsupported operand type(s) for +: 'int' and 'Err'

In [12]:
# import sys

In [10]:
# print("D")
# print("A")

# try:
#     raise ValueError()
# except Exception as e:
#     tb = e.__traceback__
# #     print(len([tb.tb_next while tb is not None])
# #     trace = []
# #     while tb is not None:
# #         trace.append({
# #             "filename": tb.tb_frame.f_code.co_filename,
# #             "name": tb.tb_frame.f_code.co_name,
# #             "lineno": tb.tb_lineno
# #         })
# #         tb = tb.tb_next


# #     print(sys.exc_info()[-1].tb_lineno)

In [11]:
# n = 0
# while tb is not None:
#     tb = tb.tb_next
#     n+=1

In [12]:
# n

In [13]:
# trace

In [14]:
import traceback
import sys

In [15]:
# try:
#     raise ValueError()
# except Exception as e:
#     print(sys.exception())
# #     print()


# # #     traceback.print_stack()


# # #     tb = e.__traceback__
# # #     traceback.print_tb(type(e), e, tb)

In [16]:
# print("\n".join(['def user_func_process_batch_47f8c8f3db3a30695a28e4a51e44916669ac3d111924cb614181c64b2c3b8323_bd42b6607712b09a0b187e3c67030f772c5dcd3c4152e4abebcb3a9f0a8259e4(batch):',
#            '', '    def process_batch(batch):', "        print(f'starting batch {batch}')",
#            '        from time import sleep', '        sleep(1)', '        [1, 2, 3][5]',
#            "        print('done')", '        return batch + 1', '    result = process_batch(batch=batch)',
#            '    return result']))